In [1]:
#Importing dependancies
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import psycopg2 as ps
from psycopg2 import sql, connect
from getpass import getpass

In [2]:
#Importing Sklearn
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [3]:
#Connecting to AWS
import getpass
engine = ps.connect(
    database="postgres",
    user= input("Username: "),
    password= getpass.getpass("Password: "),
    host="highered-db.cx2ll8zuuepz.us-east-2.rds.amazonaws.com",
    port='5432'
)

In [4]:
cr = engine.cursor()
cr.execute('SELECT * FROM highered_data_table;')
tmp = cr.fetchall()

In [5]:
#Converting to Pandas DataFrame
df = pd.DataFrame(tmp, columns= ['Year','UNITID','FACSTAT','ARANK','HRTOTLT','HRTOTLM','HRTOTLW','INSTNM','CITY','STABBR','ZIP',
                                    'CONTROL','HLOFFER','INSTSIZE','LONGITUD','LATITUDE'])
df

,Year,UNITID,FACSTAT,ARANK,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
0,2019,100654,0,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
1,2019,100654,10,0,242,131,111,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
2,2019,100654,10,1,49,37,12,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
3,2019,100654,10,2,50,34,16,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
4,2019,100654,10,3,98,46,52,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192599,2020,232618,0,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192600,2020,232618,10,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192601,2020,232618,10,6,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100
192602,2020,232618,40,0,27,0,27,Centra College,Lynchburg,VA,24501,2,5,1,-79.174101,37.405100


In [7]:
# df_test = df.drop_duplicates()
df_test = df.drop(['FACSTAT', 'ARANK'], axis=1)
df_test.sample(10)

,Year,UNITID,HRTOTLT,HRTOTLM,HRTOTLW,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
32395,2019,153472,56,28,28,Indian Hills Community College,Ottumwa,IA,52501,1,3,2,-92.394736,41.045105
166917,2020,176354,66,20,46,Southwest Mississippi Community College,Summit,MS,39666,1,3,2,-90.443295,31.295121
191793,2020,230959,34,24,10,Middlebury College,Middlebury,VT,5753,2,9,2,-73.176068,44.009117
153629,2020,145600,68,34,34,University of Illinois Chicago,Chicago,IL,60607,1,9,5,-87.650503,41.871837
68441,2021,167260,7,1,6,Nichols College,Dudley,MA,1571,2,7,2,-71.930283,42.044033
131778,2020,101736,74,35,39,Northwest-Shoals Community College,Muscle Shoals,AL,35661,1,3,2,-87.678089,34.739788
33046,2021,117168,5,2,3,Laguna College of Art and Design,Laguna Beach,CA,92651,2,7,1,-117.772564,33.561448
94159,2021,197708,12,6,6,Yeshiva University,New York,NY,10033,2,9,3,-73.929873,40.850605
41593,2021,132842,29,11,18,Albizu University-Miami,Miami,FL,33172,2,9,1,-80.356315,25.794178
143232,2020,125028,4,3,1,Ventura College,Ventura,CA,93003,1,3,4,-119.230604,34.277393


In [8]:
#Dropping duplicates
df_test.drop_duplicates(inplace=True)

In [9]:
df_test1 = df_test.groupby(['UNITID', 'Year'])['HRTOTLT'].sum().unstack(fill_value=0)
#.unstack(fill_value=0)
df_test1.head(10)

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
100751,6746,6708,6928
100760,137,127,119
100812,274,256,292
100830,865,1011,1004


In [10]:
#Creating dataframe with all other info
df_test2 = df_test[['UNITID','INSTNM','CITY','STABBR','ZIP','CONTROL','HLOFFER','INSTSIZE','LONGITUD','LATITUDE']]

In [11]:
#Dropping duplicates from df2
df_test2.drop_duplicates(inplace=True)
df_test2.sample(5)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,UNITID,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
13987,433387,Western Governors University,Salt Lake City,UT,84107,2,7,5,-111.869752,40.684810
5678,110574,California State University-East Bay,Hayward,CA,94542,1,9,4,-122.054135,37.657694
32714,115931,Institute for Business and Technology,Santa Clara,CA,95051,3,3,1,-121.969541,37.368642
75582,200004,Western Carolina University,Cullowhee,NC,28723-9646,1,9,4,-83.186258,35.308979
24507,101675,Miles College,Fairfield,AL,35064,2,5,2,-86.908751,33.482442


In [12]:
df_test1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3756 entries, 100654 to 497286
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   2019    3756 non-null   int64
 1   2020    3756 non-null   int64
 2   2021    3756 non-null   int64
dtypes: int64(3)
memory usage: 117.4 KB


In [13]:
df_test1.dropna()

Year,2019,2020,2021
UNITID,,,
100654,874,1176,1027
100663,9874,9961,10208
100690,117,100,120
100706,1578,1514,1534
100724,1026,996,970
...,...,...,...
496681,0,0,10
496991,0,0,7
497046,0,0,112


In [14]:
#Using drop() to delete rows based on column value
df_test1.drop(df_test1[df_test1[2019] == 0].index, inplace = True)

In [15]:
df3 = pd.merge(left=df_test1, right=df_test2, left_on='UNITID', right_on='UNITID')
df3.shape
df3.sample(5)

,UNITID,2019,2020,2021,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE
1059,139199,553,561,651,Brenau University,Gainesville,GA,30501,2,9,2,-83.821513,34.303117
5151,242723,78,80,67,Inter American University of Puerto Rico-Schoo...,San Juan,PR,00936-8351,2,9,1,-66.072656,18.428145
3860,211024,56,62,44,Clarks Summit University,South Abington Township,PA,18411-1250,2,9,1,-75.685472,41.495062
4657,230764,12253,12185,12377,University of Utah,Salt Lake City,UT,84112-9008,1,9,5,-111.846044,40.762484
4860,235699,162,168,160,Lake Washington Institute of Technology,Kirkland,WA,98034-8506,1,5,2,-122.167131,47.705077


In [16]:
#Calculating retention rate
df3["Retention_Rate"] = (df3[2021]/df3[2019] * 100).round(2).astype(float)
df3.sample(10)

,UNITID,2019,2020,2021,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,Retention_Rate
3763,207740,167,147,147,Seminole State College,Seminole,OK,74868,1,3,2,-96.704908,35.249515,88.02
2652,181330,134,124,141,Midland University,Fremont,NE,68025,2,7,2,-96.488008,41.436740,105.22
5999,482936,334,302,318,Florida Polytechnic University,Lakeland,FL,33805-8531,1,7,2,-81.845995,28.150149,95.21
5841,459268,24,24,22,South University-Virginia Beach,Virginia Beach,VA,23452,3,8,1,-76.124288,36.833600,91.67
3592,203748,514,504,552,Lorain County Community College,Elyria,OH,44035,1,5,4,-82.073549,41.413293,107.39
2787,185828,1999,2353,2383,New Jersey Institute of Technology,Newark,NJ,7102,1,9,4,-74.177113,40.741997,119.21
2206,170055,740,723,735,Grand Rapids Community College,Grand Rapids,MI,49503,1,3,4,-85.665625,42.967076,99.32
524,119331,246,233,229,Napa Valley College,Napa,CA,94558,1,3,2,-122.274381,38.274081,93.09
3599,203881,112,105,104,Marion Technical College,Marion,OH,43302,1,3,2,-83.089156,40.577017,92.86
4346,220996,6,12,14,Middle Tennessee School of Anesthesia Inc,Madison,TN,37115,2,9,1,-86.683859,36.253425,233.33


In [17]:
#Adding retention boolean
df3["Retention"] = np.where(df3['Retention_Rate'] > 99.99, True, False)
df3

,UNITID,2019,2020,2021,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,Retention_Rate,Retention
0,100654,874,1176,1027,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368,117.51,True
1,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294-0110,1,9,5,-86.799345,33.505697,103.38,True
2,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294,1,9,5,-86.799345,33.505697,103.38,True
3,100690,117,100,120,Amridge University,Montgomery,AL,36117-3553,2,9,1,-86.174010,32.362609,102.56,True
4,100690,117,100,120,Amridge University,Montgomery,AL,36117,2,9,1,-86.174010,32.362609,102.56,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6204,494737,19,19,19,Yeshiva Yesoda Hatorah Vetz Chaim,Brooklyn,NY,11211-7106,2,5,1,-73.961026,40.706959,100.00,True
6205,494843,20,16,16,Fortis College-Landover,Landover,MD,20785,3,3,1,-76.866209,38.949983,80.00,False
6206,494843,20,16,16,Fortis College-Landover,Landover,MD,20785-2223,3,3,1,-76.866209,38.949983,80.00,False
6207,494852,25,28,26,Stautzenberger College-Rockford Career College,Rockford,IL,61108,3,3,1,-89.027291,42.255903,104.00,True


In [18]:
#Dropping Retention Rate Column
df3.drop(columns=["Retention"])

,UNITID,2019,2020,2021,INSTNM,CITY,STABBR,ZIP,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,Retention_Rate
0,100654,874,1176,1027,Alabama A & M University,Normal,AL,35762,1,9,3,-86.568502,34.783368,117.51
1,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294-0110,1,9,5,-86.799345,33.505697,103.38
2,100663,9874,9961,10208,University of Alabama at Birmingham,Birmingham,AL,35294,1,9,5,-86.799345,33.505697,103.38
3,100690,117,100,120,Amridge University,Montgomery,AL,36117-3553,2,9,1,-86.174010,32.362609,102.56
4,100690,117,100,120,Amridge University,Montgomery,AL,36117,2,9,1,-86.174010,32.362609,102.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6204,494737,19,19,19,Yeshiva Yesoda Hatorah Vetz Chaim,Brooklyn,NY,11211-7106,2,5,1,-73.961026,40.706959,100.00
6205,494843,20,16,16,Fortis College-Landover,Landover,MD,20785,3,3,1,-76.866209,38.949983,80.00
6206,494843,20,16,16,Fortis College-Landover,Landover,MD,20785-2223,3,3,1,-76.866209,38.949983,80.00
6207,494852,25,28,26,Stautzenberger College-Rockford Career College,Rockford,IL,61108,3,3,1,-89.027291,42.255903,104.00


In [19]:
#Creating feature
X = df3.drop("Retention", axis=1)
X = X.drop("Retention_Rate", axis=1)
X = pd.get_dummies(X)

In [20]:
#Adding a target thought? 
target = ["Retention"]

In [21]:
#Creating a target
y = df3.loc[:, target].copy()

In [22]:
#Decribing
X.describe()

,UNITID,2019,2020,2021,CONTROL,HLOFFER,INSTSIZE,LONGITUD,LATITUDE,INSTNM_A T Still University of Health Sciences,...,ZIP_99324-1198,ZIP_99362,ZIP_99362-9267,ZIP_99507,ZIP_99507-1033,ZIP_99508,ZIP_99723,ZIP_99775,ZIP_99801,ZIP_99801-8697
count,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,...,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000,6209.000000
mean,219987.409728,773.177806,756.817362,756.163311,1.680142,6.085521,2.178934,-90.377168,37.620665,0.000161,...,0.000161,0.000322,0.000161,0.000161,0.000161,0.000322,0.000161,0.000161,0.000161,0.000161
std,104962.242784,2054.196758,2024.011535,2062.542663,0.694321,2.416705,1.140727,19.189893,5.940635,0.012691,...,0.012691,0.017946,0.012691,0.012691,0.012691,0.017946,0.012691,0.012691,0.012691,0.012691
min,100654.000000,1.000000,0.000000,0.000000,1.000000,3.000000,-2.000000,-170.742774,-14.322636,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,153296.000000,56.000000,52.000000,52.000000,1.000000,3.000000,1.000000,-97.471508,34.086353,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,195049.000000,224.000000,213.000000,212.000000,2.000000,7.000000,2.000000,-86.445263,38.900540,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,230746.000000,609.000000,590.000000,589.000000,2.000000,9.000000,3.000000,-78.585676,41.606820,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,494852.000000,36293.000000,30347.000000,35552.000000,3.000000,9.000000,5.000000,171.378129,71.324702,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(Counter(y_train['Retention']))
print(Counter(y_test['Retention']))

Counter({False: 2730, True: 1926})
Counter({False: 913, True: 640})


In [24]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
rf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
rf_model.fit(X_train, y_train)
print(Counter(y_train['Retention']))

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,
c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Counter({False: 2730, True: 1926})


In [25]:
# Calculated the balanced accuracy score
y_pred = rf_model.predict(X_test)
balanced_accuracy_score(y_test,y_pred)

c:\Users\david.bowen\Anaconda3\envs\mlenv\lib\site-packages\sklearn\utils\validation.py:1692: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  FutureWarning,


0.8234982543811611

In [26]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[789, 124],
       [139, 501]], dtype=int64)

In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.85      0.86      0.78      0.86      0.82      0.68       913
       True       0.80      0.78      0.86      0.79      0.82      0.67       640

avg / total       0.83      0.83      0.82      0.83      0.82      0.68      1553



In [28]:
#***** The below blocks are the Easy Ensemble Model, this took over 175 minutes to run *****

In [29]:
# # Train the EasyEnsembleClassifier
# from imblearn.ensemble import EasyEnsembleClassifier
# ee_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)
# ee_model.fit(X_train, y_train)
# print(Counter(y_train['Retention']))

In [30]:
# # Calculated the balanced accuracy score
# y_pred = ee_model.predict(X_test)
# balanced_accuracy_score(y_test,y_pred)

In [31]:
# # Display the confusion matrix
# cm2 = confusion_matrix(y_test, y_pred)
# cm2